Autores: Bruno Leal Fonseca & Guilherme Namen Pimenta

Repositório Github: https://github.com/Nagi0/bh-traffic-data-mining

# Data Understanding & Data Preparation

Para processar a base de registros de veículos da cidade de Belo Horizonte foi escolhido o mês de fevereiro de 2022. A base é muito grande, por isso filtramos para processar apenas o mês de fevereiro os veículos registrados na Av. do Contorno. Para tal o comando Powershell foi executado para gerar a base filtrada:  
```
Get-ChildItem "." -Filter *.json -Recurse | select -ExpandProperty FullName | %{Get-Content $_ | Out-String | ConvertFrom-Json | Where-Object {$_.ENDEREÇO -like 'Av. do Contorno*'}} | Export-Csv
```
A bse resultante possui 4.903.487 registros.

In [2]:
Sys.setlocale(locale = 'en_US.UTF-8')
sessionInfo()

Warning message in Sys.setlocale(locale = "en_US.UTF-8"):
"using locale code page other than 1252 may cause problems"


[1] "LC_COLLATE=en_US.UTF-8;LC_CTYPE=en_US.UTF-8;LC_MONETARY=en_US.UTF-8;LC_NUMERIC=C;LC_TIME=en_US.UTF-8"

R version 4.4.2 (2024-10-31 ucrt)
Platform: x86_64-w64-mingw32/x64
Running under: Windows Server 2019 x64 (build 17763)

Matrix products: default


locale:
[1] LC_COLLATE=en_US.UTF-8  LC_CTYPE=en_US.UTF-8    LC_MONETARY=en_US.UTF-8
[4] LC_NUMERIC=C            LC_TIME=en_US.UTF-8    
system code page: 1252

time zone: America/Sao_Paulo
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] digest_0.6.37     IRdisplay_1.1     utf8_1.2.4        base64enc_0.1-3  
 [5] fastmap_1.2.0     glue_1.8.0        htmltools_0.5.8.1 repr_1.1.7       
 [9] lifecycle_1.0.4   cli_3.6.3         fansi_1.0.6       vctrs_0.6.5      
[13] pbdZMQ_0.3-13     compiler_4.4.2    tools_4.4.2       evaluate_1.0.1   
[17] pillar_1.9.0      crayon_1.5.3      rlang_1.1.4       jsonlite_1.8.9   
[21] IRkernel_1.3.2    uuid_1.2-1       

In [3]:
library(tidyverse)
#Função para gerar o banco de dados
gera_banco_dados <- function() {
  
  #Carrega arquivo
  contorno <- read_csv('contorno.csv')
  #Filtra veículos com comprimento inválido e classificação indefinida
  contorno <- contorno %>% filter(`CLASSIFICAÇÃO` != 'INDEFINIDO')
  contorno <- contorno %>% filter(TAMANHO < 20)
  #Faz o parser da data
  contorno$`DATA HORA` <-
    parse_date_time(
      contorno$`DATA HORA`,
      '%d/%m/%Y %I:%M:%S %p',
      tz = "America/Sao_Paulo")
  #Realiza a transformação da velocidade em fatores
  velocidade <- c("0-40", "41-60", "61-100", ">100", "NA")
  cut_points <- c(0, 41, 61, Inf)
  i <- findInterval(contorno$`VELOCIDADE AFERIDA`, cut_points)
  v_fac <- velocidade[i]
  v_fac[is.na(contorno$`VELOCIDADE AFERIDA`)] <- velocidade[length(velocidade)]
  contorno$`VELOCIDADE AFERIDA` <- factor(v_fac, levels = velocidade)
  #Transforma valores em fatores
  contorno$CLASSIFICAÇÃO <- factor(contorno$CLASSIFICAÇÃO,ordered =FALSE)
  contorno$SENTIDO <- factor(contorno$SENTIDO,ordered =FALSE)
  contorno$FAIXA <- factor(contorno$FAIXA,ordered =FALSE)
  contorno$`ID DE ENDEREÇO` <- factor(contorno$`ID DE ENDEREÇO`,ordered =FALSE)
  
  #Transforma o comprimento do veículo em 3 faixas proporcional à classificação do veículo
  contorno <- 
    contorno %>%
    group_by(`CLASSIFICAÇÃO`) %>%
    mutate(BIN = cut(TAMANHO,3,c('PEQUENO','MÉDIO','GRANDE'))) %>%
    ungroup()
  return(contorno)
}

#Gera os registros do índices
indices <- function(dados) {
  A <- levels(dados$FAIXA)
  B <- levels(dados$`ID DE ENDEREÇO`)
  C <- levels(dados$`VELOCIDADE AFERIDA`)
  D <- levels(dados$`CLASSIFICAÇÃO`)
  E <- levels(dados$SENTIDO)
  F <- levels(dados$BIN)
  ind <-
    crossing(
      FAIXA=A,
      `ID DE ENDEREÇO`=B,
      #SENTIDO=E,
      `CLASSIFICAÇÃO`=D,
      BIN=F,
      `VELOCIDADE AFERIDA`=C
    )
  ind$ID <- 1:dim(ind)[1]
  return(ind)
}

#O preambulo é necessário para o processamento do arquivo pelo SPMF
gera_preambulo <- function(arquivo, tipos) {
  fileConn<-file(arquivo, encoding="UTF-8")
  writeLines(c("@CONVERTED_FROM_TEXT"), fileConn)
  close(fileConn)
  
  codigos <- tipos %>%
    unite("NOME", FAIXA:`VELOCIDADE AFERIDA`, remove = TRUE) %>%
    unite("CODIGO", ID, NOME, remove = TRUE, sep="=") %>%
    mutate(CODIGO=paste("@ITEM=", CODIGO, sep = ""))
  write.table(codigos,
              file = arquivo,
              sep = "",
              col.names = FALSE,
              row.names = FALSE,
              quote = FALSE,
              append=TRUE,
              fileEncoding="UTF-8")
  fileConn<-file(arquivo, "a")
  close(fileConn)
}

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


# Geração da Base

## Filtros
Foram removidos os veículos classificados como INDEFINIDOS e os veículos de comprimento superior a 20 metros.

## Agrupamentos
A velocidade aferida foi agrupada nos seguintes intervalos "0-40", "41-60", "61-100", ">100".
O comprimento dos veículos foram agrupados em 'PEQUENO','MÉDIO','GRANDE' de forma proporcial ao tipo de veículo

In [4]:
contorno <- gera_banco_dados()

Rows: 4933535 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (4): DATA HORA, CLASSIFICAÇÃO, ENDEREÇO, SENTIDO
dbl (10): ID EQP, MILESEGUNDO, FAIXA, ID DE ENDEREÇO, VELOCIDADE DA VIA, VEL...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
contorno[1:3,]

ID EQP,DATA HORA,MILESEGUNDO,FAIXA,ID DE ENDEREÇO,VELOCIDADE DA VIA,VELOCIDADE AFERIDA,CLASSIFICAÇÃO,TAMANHO,NUMERO DE SÉRIE,LATITUDE,LONGITUDE,ENDEREÇO,SENTIDO,BIN
<dbl>,<dttm>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<fct>,<fct>
193,2022-02-01 01:02:08,742,1,311,60,41-60,AUTOMÓVEL,4.1,3264,-19.9176,-43.94896,"Av. do Contorno, oposto ao nº 10812",Centro/Barro Preto,MÉDIO
193,2022-02-01 01:02:17,180,1,311,60,41-60,AUTOMÓVEL,3.9,3264,-19.9176,-43.94896,"Av. do Contorno, oposto ao nº 10812",Centro/Barro Preto,MÉDIO
193,2022-02-01 01:02:34,430,2,311,60,41-60,AUTOMÓVEL,4.7,3264,-19.9176,-43.94896,"Av. do Contorno, oposto ao nº 10812",Centro/Barro Preto,MÉDIO


# Geração do arquivo no formato SPMF
O aplicativo SPMF https://www.philippe-fournier-viger.com/spmf/ possui duas partes um preambulo contendo o identificador númerico e o nome do item e em seguida o conjunto de itensets somente contendo o identificador. Para gerar o identificador numérico, uma tabela de índice foi criada contendo todos os itens e seu respectivo código.
Em seguida os indentificadores são mesclados à base horiginal.

## Item
O item constitui-se da faixa de trânsito seguida do id do endereço e do tipo, tamanho e velocidade do veículo.

In [6]:
ind <- indices(contorno)
contorno <- contorno %>% inner_join(ind)

Joining with `by = join_by(FAIXA, `ID DE ENDEREÇO`, `VELOCIDADE AFERIDA`,
CLASSIFICAÇÃO, BIN)`


In [7]:
ind[1:3,]

FAIXA,ID DE ENDEREÇO,CLASSIFICAÇÃO,BIN,VELOCIDADE AFERIDA,ID
<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,311,AUTOMÓVEL,GRANDE,>100,1
1,311,AUTOMÓVEL,GRANDE,0-40,2
1,311,AUTOMÓVEL,GRANDE,41-60,3


# Transação
A transação é o agrupamento dos itens durante o perído de 5 minutos. Por tanto queremos verificar os itens mais comuns em um intervalo de cinco minutos

In [8]:
item_set <- contorno %>%
  group_by(day(`DATA HORA`), floor_date(`DATA HORA`, unit = "5minutes")) %>%
  summarise(
    ITEMSET = paste0(unique(ID), collapse = " "),
    .groups = 'drop') %>%
  select(ITEMSET)

In [9]:
item_set[1:3,]

ITEMSET
<chr>
8 278 548 313 283 567 13 323 58 53 368 638 448 373 188 728 458 387 807 1037 367 493 1043 903 103 457 632 382 908 943 463 733 773 413 637 927
548 298 278 8 543 584 549 568 13 558 273 554 343 323 868 593 58 628 363 103 143 638 98 368 908 413 127 102 943 637 448 923 907 1042 458 412 373 132 134
277 548 313 278 549 294 569 553 8 543 274 283 593 328 603 323 58 908 1043 637 448 453 369 458 413 123 373 103 1042 808 1062 492 102 368 367 903 638 383 728


# Gerar o arquivo SPMF
Para gerar o arquivo escrevemos o preâmbulo e as transações.

In [10]:
gera_preambulo("arquivo.txt", ind)
write(item_set$ITEMSET, "arquivo.txt", append = TRUE)

# Processamento da base
Para processar a base para e verificar quais itens mais frequêntes, o algoritmo NegFIM foi utilizado com suporte igqual a 90% 

In [34]:
frequentes <- read.csv('frequentes.csv')
colnames(frequentes) <- c("Itemset", "Suporte")
frequentes

Itemset,Suporte
<chr>,<dbl>
4_328_AUTOMÓVEL_PEQUENO_41-60,7314
3_366_AUTOMÓVEL_MÉDIO_41-60,7345
3_366_AUTOMÓVEL_MÉDIO_41-60 4_349_AUTOMÓVEL_MÉDIO_41-60,7257
3_366_AUTOMÓVEL_MÉDIO_41-60 2_350_AUTOMÓVEL_MÉDIO_41-60,7283
2_349_AUTOMÓVEL_MÉDIO_0-40,7364
2_349_AUTOMÓVEL_MÉDIO_0-40 4_349_AUTOMÓVEL_MÉDIO_41-60,7262
2_349_AUTOMÓVEL_MÉDIO_0-40 2_350_AUTOMÓVEL_MÉDIO_41-60,7276
2_349_AUTOMÓVEL_PEQUENO_0-40,7385
2_349_AUTOMÓVEL_PEQUENO_0-40 4_349_AUTOMÓVEL_MÉDIO_41-60,7280


# Processmanto de regras frequêntes
Para verificar as regras o algoritmo FPGrow association rule foi executado com os seguintes parâmetros:
Suporte| Confiança Mínima | Lift Mínimo
--|--|--|
0.75|0.9|1.16

In [35]:
regras <- read.csv('regras.csv')
colnames(regras) <- c("Regra", "Suporte", "Confiança", "Lift")
regras

Regra,Suporte,Confiança,Lift
<chr>,<dbl>,<dbl>,<dbl>
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_349_AUTOMÓVEL_PEQUENO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6065,0.9561722,1.160901
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_350_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6062,0.9560006,1.160693
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_350_AUTOMÓVEL_MÉDIO_41-60 3_349_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6067,0.9558847,1.160552
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_350_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 4_349_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6068,0.9558916,1.160561
1_349_AUTOMÓVEL_MÉDIO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_PEQUENO_0-40 3_366_AUTOMÓVEL_MÉDIO_41-60 4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6050,0.9560683,1.160775
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_349_AUTOMÓVEL_PEQUENO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6056,0.9568652,1.161743
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_349_AUTOMÓVEL_PEQUENO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_PEQUENO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6060,0.9562885,1.161043
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_349_AUTOMÓVEL_PEQUENO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 2_350_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6062,0.9563023,1.161059
1_349_AUTOMÓVEL_MÉDIO_41-60 1_349_AUTOMÓVEL_PEQUENO_0-40 1_349_AUTOMÓVEL_PEQUENO_41-60 1_350_AUTOMÓVEL_MÉDIO_41-60 3_349_AUTOMÓVEL_MÉDIO_41-60 3_366_AUTOMÓVEL_MÉDIO_41-60 ==> 1_366_AUTOMÓVEL_MÉDIO_41-60,6061,0.9565972,1.161417


# Processamento de sequencias
Para processar dos dados em sequência agrupamos os registros em três sequências de 5 minutos, o número -1 divide as sequências.

In [13]:
sequencia <- contorno %>%
  group_by(Dia = day(`DATA HORA`), Minuto = floor_date(`DATA HORA`, unit = "5minutes")) %>%
  summarise(
    ITEMSET = paste0(unique(ID), collapse = " "),
    .groups = 'drop') %>%
  group_by(Dia, floor_date(Minuto, unit = "15minutes")) %>%
  summarise(ITEMSET = paste0(ITEMSET, collapse = " -1 "), .groups = 'drop')

In [14]:
sequencia$ITEMSET = paste(sequencia$ITEMSET, "-2")
gera_preambulo("arquivoseq.txt", ind)
write(sequencia$ITEMSET, "arquivoseq.txt", append = TRUE)

# Processamento de sequências
O algoritmo utilizado foi o SPADE Paralelizado com suporte mínimo de 0.95

In [36]:
sq <- read.csv('seq.csv')
colnames(sq) <- c("Regra", "Suporte")
sq

Regra,Suporte
<chr>,<dbl>
1_328_AUTOMÓVEL_PEQUENO_41-60 -1,2607
1_349_AUTOMÓVEL_MÉDIO_41-60 -1,2595
1_349_AUTOMÓVEL_PEQUENO_0-40 -1,2624
1_349_AUTOMÓVEL_PEQUENO_41-60 -1,2667
2_328_AUTOMÓVEL_MÉDIO_41-60 -1,2623
2_328_AUTOMÓVEL_PEQUENO_41-60 -1,2612
2_349_AUTOMÓVEL_MÉDIO_0-40 -1,2655
2_349_AUTOMÓVEL_MÉDIO_41-60 -1,2677
2_349_AUTOMÓVEL_PEQUENO_0-40 -1,2650


# Processamento de regras sequências
Para processar as regras sequências foi utilizado o algoritmo TRulerGrowth com os seguintes parâmetros:
MinSup| Confiança Mínima | Tamanho da Janela
--|--|--|
0.9|0.9|3

In [37]:
rsq <- read.csv('rule_seq.txt')
colnames(rsq) <- c("Regra", "Suporte", "Confiança")
#rsq[order(rsq$Confiança, decreasing = TRUE), ]
rsq

Regra,Suporte,Confiança
<chr>,<dbl>,<dbl>
4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2468,0.9341408
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,4_349_AUTOMÓVEL_MÉDIO_41-60",2444,0.9250568
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60",2446,0.9258138
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60,4_349_AUTOMÓVEL_MÉDIO_41-60",2427,0.9186223
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60,2_349_AUTOMÓVEL_PEQUENO_41-60",2435,0.9216503
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60,2_349_AUTOMÓVEL_PEQUENO_41-60,4_349_AUTOMÓVEL_MÉDIO_41-60",2417,0.9148372
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60,2_350_AUTOMÓVEL_MÉDIO_41-60",2425,0.9178653
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_MÉDIO_41-60,3_349_AUTOMÓVEL_MÉDIO_41-60",2422,0.9167298
"4_370_AUTOMÓVEL_MÉDIO_0-40 ==> 1_349_AUTOMÓVEL_PEQUENO_41-60,2_349_AUTOMÓVEL_PEQUENO_41-60",2454,0.9288418


In [17]:
contorno %>% distinct(`ID DE ENDEREÇO`,ENDEREÇO, SENTIDO)

ID DE ENDEREÇO,ENDEREÇO,SENTIDO
<chr>,<chr>,<fct>
311,"Av. do Contorno, oposto ao nº 10812",Centro/Barro Preto
328,"Av. do Contorno, nº 10.730",Barro Preto/Centro
349,"Av. do Contorno, nº 629",Praça da Estação / Rodoviária
350,"Av. do Contorno, nº 4.045",Savassi / Sta. Efigênia
366,"Av. do Contorno, oposto ao nº 5.445",São Lucas/Savassi
370,"Av. do Contorno, oposto ao nº 4.045",Sta. Efigênia / Savassi


# Análise

Dentro do escopo da Av do Contorno no mês de fevereiro de 2022, podemos verificar os seguintes aspéctos:
## Itensets frequêntes:
* O veículos que mais são captados na via são automóveis de tamanho médio ou pequeno com velocidade entre 41 Km/h e 60 Km/h circulando nos números:
    * 4.045, sentido Savassi / Sta. Efigênia na faixa número 2;
    * 629, sentido Praça da Estação / Rodoviária nas faixas de número 2, 3 e 4;
    * oposto ao nº 5.445, sentido São Lucas/Savassi na faixa número 2.
## Regras frequêntes:
As regras, na interpretação dos autores, não foram muito produtivas. Acredita-se que o evento de um veículo se captado por um radar seja totalmente idependete dos demais. Porém a base é muito grande e talvez haja regras mais produtivas, mas para isto seria necessário um poder computacional maior. O que seria interessante é executar o algoritmo OPUS Miner que consegue obter regras com valor de lift alto, mas mesmos estas regras podem não ser interessantes pelo fato de ter um suporte muito baixo.

## Itemset sequencial
Devido o tamanho da base as regras de maior frequencia são semelhantes aos resultados dos itens frequêntes. Foi observado um relação entre o tráfego dos números 629, sentido Praça da Estação / Rodoviária e 10.730 sentido Barro Preto/Centro em sequência. Indicando, provavelmente, que o fluxo dos dois ponto convergem simultaneamente para o centro da cidade. Um outra análise é que outros endereço em sequencia geram fluxo para o número 10.730, indicando um possível concentração de fluxo de trânsito. Este ponto tem sua importância por ligar bairros populares ao centro indicando o provável fluxo de pessoas para o trabalho.

# Regras sequenciais
O resultado mostra alguns posívies pontos de trânsito intenso como o número número 4.045 que recebe transito nos dois sentidos indicando que provavelmente neste ponto o fluxo é inteso e independete do sentido.

# Análise via Chat GPT

## Itensets frequêntes:
    Prompt : Tarefa: Descrever em um texto corrido os resultados do tráfego da av do contorno de belo horizonte.
    Precondições: A primeira coluna os dados estão separados pelo caractere _ na seguinte ordem: faixa de circulação, código do endereço, tipo do veículo, porte do veículo e intervalo de velocidade aferida, podendo haver mais de um registro. A segunda coluna é a quantidade de observações. 
    Resultado obtido:
    2_350_AUTOMÓVEL_MÉDIO_41-60	7881
    2_349_AUTOMÓVEL_MÉDIO_41-60	7870
    2_349_AUTOMÓVEL_PEQUENO_41-60	7869
    4_349_AUTOMÓVEL_MÉDIO_41-60	7858
    3_349_AUTOMÓVEL_MÉDIO_41-60	7809
    2_349_AUTOMÓVEL_PEQUENO_41-60 2_349_AUTOMÓVEL_MÉDIO_41-60	7762
    2_349_AUTOMÓVEL_MÉDIO_41-60 2_350_AUTOMÓVEL_MÉDIO_41-60	7726
    4_349_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_MÉDIO_41-60	7723
    4_349_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_PEQUENO_41-60	7722
    2_349_AUTOMÓVEL_PEQUENO_41-60 2_350_AUTOMÓVEL_MÉDIO_41-60	7721
    4_349_AUTOMÓVEL_MÉDIO_41-60 2_350_AUTOMÓVEL_MÉDIO_41-60	7720
    2_366_AUTOMÓVEL_MÉDIO_41-60	7707
    1_349_AUTOMÓVEL_PEQUENO_41-60	7706
    3_349_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_PEQUENO_41-60	7697
    3_349_AUTOMÓVEL_MÉDIO_41-60 2_349_AUTOMÓVEL_MÉDIO_41-60	7689
    
    Tabela dos códigos dos endereços:
    ID DE ENDEREÇO	ENDEREÇO	SENTIDO
    <chr>	<chr>	<fct>
    311	Av. do Contorno, oposto ao nº 10812	Centro/Barro Preto
    328	Av. do Contorno, nº 10.730	Barro Preto/Centro
    349	Av. do Contorno, nº 629	Praça da Estação / Rodoviária
    350	Av. do Contorno, nº 4.045	Savassi / Sta. Efigênia
    366	Av. do Contorno, oposto ao nº 5.445	São Lucas/Savassi
    370	Av. do Contorno, oposto ao nº 4.045	Sta. Efigênia / Savassi

    Resultado: Os dados de tráfego na Av. do Contorno em Belo Horizonte revelam informações detalhadas sobre a circulação de veículos em diferentes trechos e faixas da via. Observou-se que a maior concentração de veículos foi registrada na faixa 2, no endereço nº 350 (Savassi/Sta. Efigênia), com predominância de automóveis médios trafegando no intervalo de velocidade de 41 a 60 km/h, totalizando 7.881 observações. Esse padrão também foi observado na faixa 2 do endereço nº 349 (Praça da Estação/Rodoviária), com 7.870 registros do mesmo tipo de veículo e velocidade.

    Automóveis pequenos também apresentaram volume significativo de tráfego, com destaque para a faixa 2 do endereço nº 349, onde houve 7.869 observações no mesmo intervalo de velocidade. Outros trechos da via, como o nº 366 (São Lucas/Savassi) e nº 350, também registraram volumes consideráveis de veículos médios na faixa 2, com 7.707 e 7.726 observações, respectivamente.

    Além disso, combinações de registros múltiplos indicam fluxos intensos em determinados trechos. Por exemplo, o tráfego combinado de automóveis médios nas faixas 2 e 4 do endereço nº 349 totalizou 7.723 observações, sugerindo um movimento sincronizado ou simultâneo entre essas faixas.

    No endereço nº 629 (Praça da Estação/Rodoviária), a faixa 3 teve um tráfego expressivo de automóveis médios, com 7.809 observações, enquanto a faixa 1 apresentou 7.706 registros de automóveis pequenos, demonstrando a diversidade de veículos e padrões de uso entre as faixas.

    Em síntese, os dados apontam uma predominância de automóveis médios e pequenos na faixa 2 da Av. do Contorno, especialmente nos trechos nº 350 e nº 349, com velocidades entre 41 e 60 km/h. Esses resultados evidenciam padrões consistentes de fluxo e podem subsidiar futuras intervenções para otimizar a circulação na região.

## Regras frequentes
    O Chat GPT não conseguiu gerar uma interpretação de boa qualidade das regras frequêntes. Talvez seja pela complexidade das regras geradas.

# Conclusão

Os algoritmos de mineração de dados geram uma grande quantidade de informações quando são submetidos a bases muito grandes e quando há uma fraca correlação dos itens com suas transações. No caso a mineração de itens frequêntes serviu mais para ter uma visão geral das informações. No caso de dados de trânsito a mineração de itens frequêntes tem vantagem para caracterizar os principais veículos das vias e sua velocidade. Acredita-se que a análise de itens sequenciais frequentes gere mais informações válidas como pontos de trânsito de mesma intesidade nas duas vias e fluxo contínuo para o Centro da Cidade.